#### 네트워크 시각화
- 기본적으로 torch 는 print('모델')하면 텍스트로 모델 구조 보여준다
- 그외 방법으로 Torchviz, Netron, TorchView 등의 모듈이 있다.
- https://stackoverflow.com/questions/52468956/how-do-i-visualize-a-net-in-pytorch/60253769#60253769

##### Torchviz
- 대략적으로 보려면 다음과 같이 -> make_dot(y.mean(), params=dict(model.named_parameters()))
- autograd save 까지 보려면 다음과 같이 한다. make_dot(y.mean(), params=dict(model.named_parameters()), show_attrs=True, show_saved=True)
- 그래프 저장 관련 문의
  - https://github.com/szagoruyko/pytorchviz/issues/2

In [4]:
import torch
import torch.nn as nn
from torchviz import make_dot

# 예시 모듈 정의
class SubModule(nn.Module):
    def __init__(self, input_size, output_size):
        super(SubModule, self).__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x

# 메인 모듈 정의
class MainModule(nn.Module):
    def __init__(self):
        super(MainModule, self).__init__()

        # SubModule을 포함하는 예시
        self.submodule1 = SubModule(input_size=10, output_size=5)
        self.submodule2 = SubModule(input_size=5, output_size=3)

    def forward(self, x):
        x = self.submodule1(x)
        x = self.submodule2(x)
        return x

# 모델 인스턴스 생성
main_model = MainModule()
print('main_model---->',main_model)

# 모델 사용 예시
input_data = torch.randn(1, 10)  # 입력 데이터 생성
output = main_model(input_data)   # 모델에 입력 데이터 전달

print('dict(main_model.named_parameters())---->', dict(main_model.named_parameters()))
print('\n')

make_dot(output.mean(), params=dict(main_model.named_parameters())).render("output_", format="png")

print('\n')

# 모듈 및 서브모듈에 대한 속성 접근
print("Main Module:")
print(main_model)

print("\nSubmodule 1:")
print(main_model.submodule1)

print("\nSubmodule 2:")
print(main_model.submodule2)

main_model----> MainModule(
  (submodule1): SubModule(
    (linear): Linear(in_features=10, out_features=5, bias=True)
    (activation): ReLU()
  )
  (submodule2): SubModule(
    (linear): Linear(in_features=5, out_features=3, bias=True)
    (activation): ReLU()
  )
)
dict(main_model.named_parameters())----> {'submodule1.linear.weight': Parameter containing:
tensor([[ 0.0584,  0.2024,  0.0744,  0.1123, -0.2596,  0.0081, -0.1063, -0.2419,
         -0.0972,  0.1196],
        [ 0.1679, -0.0120,  0.1341, -0.0239,  0.2706, -0.0184,  0.1535,  0.2570,
          0.3005,  0.2690],
        [-0.1113,  0.1624,  0.1905, -0.1317,  0.0950, -0.0105,  0.0093,  0.1520,
         -0.0597, -0.2505],
        [-0.1164,  0.0367, -0.2348, -0.2752, -0.0581, -0.1413,  0.1665,  0.1227,
         -0.0771,  0.2088],
        [ 0.2613, -0.1239, -0.1983, -0.0537,  0.0058,  0.0162, -0.2227, -0.1866,
          0.2778,  0.1969]], requires_grad=True), 'submodule1.linear.bias': Parameter containing:
tensor([-0.2914, -0.0185

#### 시행 착오
모델을 객체로 불러서 시각화 하면 되었었는데(아래예시)  
las_inference_model = model_builder.build(self.m_model_config)  
  
밑의 과정을 거치니 마지막 노드만 출력이 되었다.  
las_inference_model = load_checkpoint(model_load_path, las_inference_model)  
las_inference_model.to(self.m_pytorch_device)  
las_inference_model.eval()  
  
load_checkpoint() 함수에서 문제가 발생한 것인지, 아니면, eval()이 문제인지(train()일때는 수행 되었었다)
원인 파악이 안되었다.

#### Torchview
- input_size를 입력으로 받아서, 본인의 모델과 같이 여러 서브 모듈을 가지고 있는 모듈은 어떻게 input_size로 줘야 할지 파악하지 못했다.
- 모델 저장 관련: https://github.com/mert-kurttutan/torchview/issues/35

In [6]:
from torchview import draw_graph

model = MLP()
batch_size = 2
# device='meta' -> no memory is consumed for visualization
model_graph = draw_graph(model, input_size=(batch_size, 128), device='meta')
model_graph.visual_graph

NameError: name 'MLP' is not defined